# 🚀 XAU-SNIPER Training (GitHub + Drive Workflow)

Este notebook integra tu código en GitHub con tus datos/almacenamiento en Google Drive.

### ⚡ ANTI-DESCONEXIÓN (IMPORTANTE)
Para evitar que Colab se desconecte durante las 3-4 horas de entrenamiento:
1. Presiona `Ctrl + Shift + I` (o F12) para abrir la consola del navegador.
2. Ve a la pestaña **Console**.
3. Pega el siguiente código y dale Enter:
```javascript
function ClickConnect(){
    console.log("Manteniendo conexión activa... 🔄"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60000)
```
Esto hará un "clic" virtual cada minuto para mantener la sesión viva.

### Requisitos:
1. **GitHub:** El código se descarga solo.
2. **Google Drive:** 
    - Crea carpeta `GLDIA_DATA`.
    - Sube `xauusd_m1.csv`.


In [ ]:
# 1. Instalar Librerías
!pip install stable-baselines3[extra] sb3-contrib pandas numpy ta shimmy gymnasium matplotlib seaborn

In [ ]:
# 2. Clonar Repositorio (Código Reciente)
import os
# Asegurar que estamos en la raiz antes de clonar para evitar carpetas anidadas
try:
    %cd /content
except:
    pass

if os.path.exists('gldia'):
    %cd gldia
    !git pull
else:
    !git clone https://github.com/alejoamorocho/gldia.git
    %cd gldia

In [ ]:
# 3. Conectar Google Drive (para Datos y Outputs)
from google.colab import drive
drive.mount('/content/drive')

# Rutas
DRIVE_ROOT = '/content/drive/MyDrive/GLDIA_DATA'
REPO_DATA = '/content/gldia/data'
REPO_OUTPUTS = '/content/gldia/outputs'

# Crear carpeta en Drive si no existe
if not os.path.exists(DRIVE_ROOT):
    os.makedirs(DRIVE_ROOT)
    print(f"Carpeta creada en Drive: {DRIVE_ROOT}. ¡Sube aquí tus CSVs gigantes!")

# Enlazar Datos (Symlink)
# El código busca datos en 'data/'. Haremos que 'data/' tome los archivos de Drive.
# Específicamente los archivos M1 que no están en GitHub.
import shutil

files_to_link = ['xauusd_m1.csv', 'xauusd_m5.csv', 'xauusd_m15.csv']

for f in files_to_link:
    src = os.path.join(DRIVE_ROOT, f)
    dst = os.path.join(REPO_DATA, f)
    if os.path.exists(src):
        if os.path.exists(dst):
            os.remove(dst)
        os.symlink(src, dst)
        print(f"Enlazado: {f} -> Drive")
    else:
        print(f"Aviso: {f} no encontrado en Drive ({DRIVE_ROOT}). Si lo necesitas, súbelo.")

# Enlazar Outputs (Para que los modelos se guarden en Drive y no se pierdan)
# Simplemente pasaremos --output_dir al script de entrenamiento
drive_output_path = os.path.join(DRIVE_ROOT, 'outputs')

In [ ]:
# 4. AUTO-RESUME: Buscar último checkpoint para reanudar
import glob

def find_latest_checkpoint(base_dir):
    # Buscar todas las carpetas 'run_*'
    runs = glob.glob(os.path.join(base_dir, "run_*"))
    if not runs:
        return None
    
    # Ordenar por fecha (nombre de carpeta)
    latest_run = sorted(runs)[-1]
    print(f"Run más reciente detectada: {latest_run}")
    
    # Buscar modelo 'final_model.zip' o 'best_model.zip'
    final_model = os.path.join(latest_run, "models", "final_model.zip")
    best_model = os.path.join(latest_run, "models", "best_model.zip")
    
    if os.path.exists(final_model):
        return final_model
    elif os.path.exists(best_model):
        return best_model
    else:
        return None

latest_ckpt = find_latest_checkpoint(drive_output_path)
resume_cmd = ""

if latest_ckpt:
    print(f"✅ ENCONTRADO CHECKPOINT: {latest_ckpt}")
    print("Reanudando entrenamiento desde ahí...")
    resume_cmd = f"--resume \"{latest_ckpt}\""
else:
    print("✨ No se encontraron checkpoints previos. Iniciando entrenamiento DESDE CERO.")


In [ ]:
# 5. 🐉 EJECUTAR ENTRENAMIENTO
# --timesteps 5000000 (5M)
# --save_freq 100000 (Guardar backup cada 100k pasos en Drive)
# --output_dir (Guardar directo en Drive)

!python training/train.py --timesteps 5000000 --save_freq 100000 --output_dir "{drive_output_path}" {resume_cmd}

In [ ]:
# 6. 📊 VISUALIZAR RESULTADOS (PnL Curve)
# Ejecuta esta celda cuando termine (o si interrumpes) para ver cómo le fue.
import pandas as pd
import matplotlib.pyplot as plt
import glob

# Buscar el archivo de trades más reciente en Drive
all_trades_files = glob.glob(os.path.join(drive_output_path, "run_*", "evaluation_trades*.csv"))

if all_trades_files:
    latest_file = sorted(all_trades_files)[-1]
    print(f"Analizando archivo: {latest_file}")
    
    df = pd.read_csv(latest_file)
    
    if not df.empty:
        # Calcular PnL Acumulado
        df['cumulative_pnl'] = df['pnl'].cumsum()
        
        plt.figure(figsize=(12, 6))
        plt.plot(df['cumulative_pnl'], label='Cumulative PnL', color='gold')
        plt.title('Curva de Equidad (Backtest)', fontsize=16)
        plt.xlabel('Número de Trade')
        plt.ylabel('PnL ($)')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()
        
        print("\nEstadísticas Rápidas:")
        print(f"Total Trades: {len(df)}")
        print(f"Total PnL: ${df['pnl'].sum():.2f}")
        print(f"Win Rate: {(len(df[df['pnl']>0])/len(df)*100):.2f}%")
    else:
        print("El archivo está vacío.")
else:
    print("No se encontraron archivos de trades en Drive aún.")